In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))

import shutil
import datetime
import numpy as np
import pandas as pd

from library import common as cm
from library import regression_aux as raux

In [ ]:
%run setup.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_SEED)

In [ ]:
%run Load_Clean_aux.py
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

In [ ]:
print(f'The number of samples after loading is : {df.shape[0]}')

regr_dir = res_dir + 'Regression/'
print(f'Results are saved at:\n{regr_dir}')

In [ ]:
if os.path.exists(regr_dir):
    shutil.rmtree(regr_dir)
os.makedirs(regr_dir)
shutil.copy('setup.py', regr_dir);

### NO permute

In [ ]:
def save_tables(res, dirs):
    if res is not None:
        res['df_coef'].to_csv(f'{dirs}coef.csv')
        res['df_fit_std'].to_csv(f'{dirs}std.csv')

#### No hedge

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}No_Hedge/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    zero = np.array([0.]*len(df))
    zero = pd.Series(zero, index=df.index)

    cm.store_pnl(df, zero, pnl_path)
    save_tables(None, sub_regr_dir)

#### Baseline: Black-Scholes 

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}BS_Benchmark/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    cm.store_pnl(df, df['delta_bs'], pnl_path)
    save_tables(None, sub_regr_dir)

#### Using fixed constants 0.9, 1.1

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Fixed_Constants/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    bl_c = df['cp_int'] == 0
    delta = 0.9 * df['delta_bs']
    delta[~bl_c] = 1.1 * df.loc[~bl_c, 'delta_bs']
    
    cm.store_pnl(df, delta, pnl_path)
    save_tables(None, sub_regr_dir)

#### Delta-only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_only/'
    kwargs = {'vix': VIX, 
          'features': ['delta_bs'], 
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': False,
          'agg_side': False,
          'leverage': True}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, dirs=sub_regr_dir)
    res['df_leve'].to_csv(f'{sub_regr_dir}leverage.csv')

#### Delta-Vega

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega/'
    kwargs = {'vix': VIX, 
          'features': ['delta_bs', 'vega_n'], 
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': False,
          'agg_side': False}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Gamma

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Gamma/'
    kwargs = {'vix': VIX, 
          'features': ['delta_bs', 'gamma_n'], 
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': False,
          'agg_side': False}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vanna/'
    kwargs = {
        'vix': VIX,
        'features': ['delta_bs', 'vanna_n'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'delta_coeff_1': False
    }

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Gamma-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Gamma_Vanna/'
    kwargs = {
        'vix': VIX,
        'features': ['delta_bs', 'gamma_n', 'vanna_n'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'delta_coeff_1': False
    }

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vega-Gamma

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Gamma/'
    kwargs = {
        'vix': VIX,
        'features': ['delta_bs', 'vega_n', 'gamma_n'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'delta_coeff_1': False
    }

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vega-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Vanna/'
    kwargs = {
        'vix': VIX,
        'features': ['delta_bs', 'vega_n', 'vanna_n'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'delta_coeff_1': False
    }

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vega-Gamma-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Gamma_Va/'
    kwargs = {
        'vix': VIX,
        'features': ['delta_bs', 'vega_n', 'gamma_n', 'vanna_n'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'delta_coeff_1': False
    }

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Vega-only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Vega_only/'
    kwargs = {'vix': VIX, 
          'features': ['vega_n'], 
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': True,
          'agg_side': False}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Vanna-only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Vanna_only/'
    kwargs = {'vix': VIX, 
          'features': ['vanna_n'],   
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': True,
          'agg_side': False}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Gamma-only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Gamma_only/'
    kwargs = {'vix': VIX, 
          'features': ['gamma_n'], 
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': True,
          'agg_side': False}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Bias only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Bias/'
    df['bias'] = 1.
    kwargs = {'vix': VIX, 
          'features': ['bias'],   
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': True,
          'agg_side': False}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Hull-White

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Hull_White/'
    kwargs = {'vix': VIX, 
          'features': ['vega_s', 'delta_vega_s', 'delta2_vega_s'], 
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': True,
          'agg_side': False}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Hull_White_relaxed/'
    kwargs = {'vix': VIX, 
          'features': ['delta_bs', 'vega_s', 'delta_vega_s', 'delta2_vega_s'], 
          'max_period': max_period, 
          'sub_res': f'{sub_regr_dir}pnl/',
          'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
          'df': df,
          'delta_coeff_1': False,
          'agg_side': False}

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

### Permute

#### No Hedge

In [ ]:
if PERMUTE:
    df_permute = df.copy()
    for i in range(NUM_PERMUTE):
        df_permute = cm.rolling_permute(df_permute, random_seed=i + 100)
        sub_res = f'{regr_dir}No_Hedge/pnl/'
        os.makedirs(sub_res, exist_ok=True)
        pnl_path = f'{sub_res}pnl{i}.csv'
        zero = np.array([0.]*len(df_permute))
        zero = pd.Series(zero, index=df_permute.index)
        cm.store_pnl(df_permute, zero, pnl_path)
    del df_permute

#### BS 

In [ ]:
if PERMUTE:
    df_permute = df.copy()
    for i in range(NUM_PERMUTE):
        df_permute = cm.rolling_permute(df_permute, random_seed=i + 100)
        sub_res = f'{regr_dir}BS_Benchmark/pnl/'
        os.makedirs(sub_res, exist_ok=True)
        pnl_path = f'{sub_res}pnl{i}.csv'
        cm.store_pnl(df_permute, df_permute['delta_bs'], pnl_path)
    del df_permute

#### Delta-Vega-Vanna

In [ ]:
if PERMUTE:
    df_permute = df.copy()
    sub_regr_dir = f'{regr_dir}Delta_Vega_Vanna/'
    for i in range(NUM_PERMUTE):
        df_permute = cm.rolling_permute(df_permute, random_seed=i + 100)
        kwargs = {'vix': VIX, 
              'features': ['delta_bs', 'vega_n', 'vanna_n'], 
              'max_period': max_period, 
              'sub_res': f'{sub_regr_dir}pnl/',
              'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
              'df': df_permute,
             'delta_coeff_1': False,
                 'agg_side': False}
        
        res = raux.run_store_lin(**kwargs)